In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms as T
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import nrrd
import pandas as pd
import os
import sys

sys.path.append("/mnt/raid/C1_ML_Analysis/source/us-famli-pl/src/")

from loaders import ultrasound_dataset as usd
from nets import classification as usn


In [ ]:
 # Datasets and loaders
# group.add_argument('--mount_point', type=str, default="./", help="Mount point for the data")
# group.add_argument('--csv_train', type=str, required=True, help="Training data csv file path")
# group.add_argument('--csv_valid', type=str, required=True, help="Validation data csv file path")
# group.add_argument('--csv_test', type=str, required=True, help="Test data csv file path")
# group.add_argument('--img_column', type=str, default="file_path")
# group.add_argument('--tag_column', type=str, default="tag")
# group.add_argument('--frame_column', type=str, default="frame_index")
# group.add_argument('--frame_label', type=str, default="annotation_label")
# group.add_argument('--id_column', type=str, default="annotation_id")
# group.add_argument('--batch_size', type=int, default=4, help="Batch size for the train dataloaders")
# group.add_argument('--num_frames', type=int, default=64, help="Number of frames to sample from each cine")
# group.add_argument('--num_workers', type=int, default=1)
# group.add_argument('--prefetch_factor', type=int, default=2)
# group.add_argument('--drop_last', type=int, default=False)


args_dict = {
    "mount_point": "/mnt/raid/C1_ML_Analysis/",
    "csv_train": "/mnt/raid/C1_ML_Analysis/CSV_files/c3_blindsweep_annotation_labels_merged_train_train.csv",
    "csv_valid": "/mnt/raid/C1_ML_Analysis/CSV_files/c3_blindsweep_annotation_labels_merged_train_test.csv",
    "csv_test": "/mnt/raid/C1_ML_Analysis/CSV_files/c3_blindsweep_annotation_labels_merged_test.csv",
    "img_column": "file_path",
    "tag_column": "tag",
    "frame_column": "frame_index",
    "frame_label": "annotation_label",
    "id_column": "annotation_id",
    "batch_size": 1,
    "num_frames": 64,
    "num_workers": 1,
    "prefetch_factor": 1,
    "drop_last": False
}

batch_size = 2
dm = usd.USAnnotatedBlindSweepDataModule(
        **args_dict
    )
dm.setup()

In [ ]:
train_dl = dm.train_dataloader()
valid_dl = dm.val_dataloader()
test_dl = dm.test_dataloader()

In [ ]:
import torch
from torchvision.utils import make_grid
import plotly.express as px

def plot_tensor_grid(tensor, nrow=4, title="Image Grid", scale=1.0):
    """
    Uses torchvision's make_grid to arrange images into a grid, then displays it with Plotly.

    Args:
        tensor: PyTorch tensor of shape [B, N, C, T, H, W]
        nrow: Number of images per row in the grid
        title: Plot title
    """
    B, N, C, T, H, W = tensor.shape
    tensor = tensor.permute(0,1,3,2,4,5)  # [B, N, T, C, H, W]

    # Flatten batch and image dimensions
    tensor = tensor.view(B * N * T, C, H, W)

    # Create image grid
    grid = make_grid(tensor, nrow=nrow, padding=2, normalize=True, scale_each=True)

    # Convert to numpy for Plotly
    grid_np = grid.permute(1,2,0).cpu().numpy()  # shape [H, W] since it's grayscale

    # Determine scaled dimensions
    height, width, C = grid_np.shape
    display_height = int(height * scale)
    display_width = int(width * scale)

    # Plot with Plotly
    fig = px.imshow(grid_np)
    fig.update_layout(
        title=title,        
        width=display_width,
        height=display_height
    )
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()


In [ ]:
def plot_tensor_grid_batch(tensor, nrow=4, title="Image Grid", scale=1.0):
    """
    Uses torchvision's make_grid to arrange images into a grid, then displays it with Plotly.

    Args:
        tensor: PyTorch tensor of shape [B, C, H, W]
        nrow: Number of images per row in the grid
        title: Plot title
    """
    B, C, H, W = tensor.shape

    # Create image grid
    grid = make_grid(tensor, nrow=nrow, padding=2, normalize=True, scale_each=True)

    # Convert to numpy for Plotly
    grid_np = grid.permute(1,2,0).cpu().numpy()  # shape [H, W] since it's grayscale

    # Determine scaled dimensions
    height, width, C = grid_np.shape
    display_height = int(height * scale)
    display_width = int(width * scale)

    # Plot with Plotly
    fig = px.imshow(grid_np)
    fig.update_layout(
        title=title,        
        width=display_width,
        height=display_height
    )
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.show()

In [ ]:
batch = next(iter(train_dl))

# plot_tensor_grid(batch[0], nrow=4, title='Image Grid')

In [ ]:
# for batch in train_dl:
#     img, labels, tags = batch

# for batch in valid_dl:
#     img, labels, tags = batch

# for batch in test_dl:
#     img, labels, tags = batch
    

In [ ]:
# group = parent_parser.add_argument_group("Fetal Anatomy time aware classification Model")

# group.add_argument("--lr", type=float, default=1e-4)
# group.add_argument("--betas", type=tuple, default=(0.9, 0.999), help='Betas for Adam optimizer')
# group.add_argument('--weight_decay', help='Weight decay for optimizer', type=float, default=1e-5)

# # Image Encoder parameters 
# group.add_argument("--spatial_dims", type=int, default=2, help='Spatial dimensions for the encoder')
# group.add_argument("--in_channels", type=int, default=3, help='Input channels for encoder')
# group.add_argument("--features", type=int, default=1280, help='Number of output features for the encoder')
# group.add_argument("--time_dim_train", type=int, nargs="+", default=(16, 128), help='Range of time dimensions for training')
# group.add_argument("--n_chunks_e", type=int, default=2, help='Number of chunks in the encoder stage to reduce memory usage')
# group.add_argument("--n_chunks", type=int, default=16, help='Number of outputs in the time dimension, this will determine the first dimension of the 2D positional encoding')
# group.add_argument("--num_heads", type=int, default=8, help='Number of heads for multi_head attention')

# group.add_argument("--embed_dim", type=int, default=128, help='Embedding dimension')        
# group.add_argument("--dropout", type=float, default=0.1, help='Dropout rate')
# group.add_argument("--tags", type=int, default=17, help='Number of sweep tags for the sequences, this will determine the second dimension of the 2D positional encoding')

# group.add_argument("--num_classes", type=int, default=3, help='Number of output classes for the model')        
# group.add_argument("--class_weights", nargs="+", default=None, type=float, help='Class weights for the loss function')

n_tags = 16

args_model = {
    "lr": 1e-4,
    "in_channels": 3,
    "features": 1280,
    "n_chunks_e": 2,
    "n_chunks": 16,
    "num_heads": 8,
    "embed_dim": 128,
    "dropout": 0.25,
    "tags": n_tags,
    "num_classes": 5,
    "class_weights": [0.1, 0.7, 4, 30, 200]
}

model = usn.FlyToClassification(**args_model).cuda()

X, sweep_tags, Y = batch
X_hat = model(X.cuda(), sweep_tags.cuda())
# x = 

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
df_train = pd.read_csv(args_dict['csv_train'])

cl = df_train['annotation_label'].apply(lambda x: dm.train_ds.frame_labels_dict[x])
cl.value_counts()



In [ ]:
cl_r = list(cl)
cl_r += [0] * (2*len(cl_r))
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(cl_r), y=cl_r)

In [ ]:
class_weights

In [ ]:
np.unique(cl)

In [ ]:
cl.value_counts()

In [ ]:
df_merged = pd.read_csv(os.path.join(args_dict['mount_point'], 'CSV_files/c3_blindsweep_annotation_labels_merged.csv'))
df_merged['annotation_id'].value_counts()

In [ ]:
 # Datasets and loaders
# group.add_argument('--mount_point', type=str, default="./", help="Mount point for the data")
# group.add_argument('--csv_train', type=str, required=True, help="Training data csv file path")
# group.add_argument('--csv_valid', type=str, required=True, help="Validation data csv file path")
# group.add_argument('--csv_test', type=str, required=True, help="Test data csv file path")
# group.add_argument('--img_column', type=str, default="file_path")
# group.add_argument('--tag_column', type=str, default="tag")
# group.add_argument('--frame_column', type=str, default="frame_index")
# group.add_argument('--frame_label', type=str, default="annotation_label")
# group.add_argument('--id_column', type=str, default="annotation_id")
# group.add_argument('--batch_size', type=int, default=4, help="Batch size for the train dataloaders")
# group.add_argument('--num_frames', type=int, default=64, help="Number of frames to sample from each cine")
# group.add_argument('--num_workers', type=int, default=1)
# group.add_argument('--prefetch_factor', type=int, default=2)
# group.add_argument('--drop_last', type=int, default=False)


# args_dict = {
#     "mount_point": "/mnt/raid/C1_ML_Analysis/",
#     "csv_train": "/mnt/raid/C1_ML_Analysis/CSV_files/c3_blindsweep_annotation_labels_merged_train_train.csv",
#     "csv_valid": "/mnt/raid/C1_ML_Analysis/CSV_files/c3_blindsweep_annotation_labels_merged_train_test.csv",
#     "csv_test": "/mnt/raid/C1_ML_Analysis/CSV_files/c3_blindsweep_annotation_labels_merged_test.csv",
#     "img_column": "file_path",    
#     "frame_column": "frame_index",
#     "frame_label": "annotation_label",
#     "id_column": "annotation_id",
#     "batch_size": 2,
#     "num_frames": 64,
#     "num_workers": 1,
#     "prefetch_factor": 1,
#     "use_extra_ac": 0,
#     "drop_last": False
# }

# batch_size = 2
# dm = usd.USAnnotatedBlindSweep2DFramesDataModule(
#         **args_dict
#     )
# dm.setup()


# batch = next(iter(dm.train_dataloader()))
# batch[0].shape




In [ ]:
# for batch in dm.train_dataloader():
#     img, labels = batch

# for batch in dm.val_dataloader():
#     img, labels = batch

# for batch in dm.test_dataloader():
#     img, labels = batch
#     img, labels = batch

In [ ]:
class NoiseLevelTransform:
    """
    Adds random Gaussian noise to `x`, scaled per element along a chosen dimension.
    """
    def __init__(self, dim=0, min_scale=0.0, max_scale=0.5, noise_std=1.0):
        """
        Args:
            dim (int): Dimension along which to vary noise level (e.g. 0=batch, 1=frames, 2=channels).
            min_scale (float): Minimum blending factor for noise (0=no noise).
            max_scale (float): Maximum blending factor for noise (1=full noise).
            noise_std (float): Standard deviation of Gaussian noise.
        """
        self.dim = dim
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.noise_std = noise_std

    def __call__(self, x):
        noise = torch.randn_like(x) * self.noise_std

        # Create broadcastable noise scaling
        scale_shape = [1] * x.ndim
        scale_shape[self.dim] = x.shape[self.dim]
        scale = torch.empty(scale_shape, device=x.device).uniform_(self.min_scale, self.max_scale)

        return (1.0 - scale) * x + scale * noise

In [ ]:
# plot_tensor_grid_batch(NoiseLevelTransform()(batch[0]), nrow=4, title='Image Grid')

In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"
df_c3 = pd.read_csv(os.path.join(mount_point,'CSV_files/c3_instance_analysis_dataset.csv'))
df_c3.columns

In [ ]:
df_c3.query('cine == 1')['tag'].value_counts().to_dict()

In [ ]:
# df_c3_flyto = df_c3.query('split == "train" and cine == 1')

# df_c3_flyto = df_c3_flyto[df_c3_flyto['tag'].isin(['AC', 'BPD', 'FL', 'TCD', 'HL', 'CRL'])]
# df_c3_flyto_dcm = df_c3_flyto[df_c3_flyto['file_path'].str.contains('.dcm', na=False)]
# df_c3_flyto_mp4 = df_c3_flyto[~df_c3_flyto['file_path'].str.contains('.dcm', na=False)]

# df_c3_flyto_dcm.to_csv(os.path.join(mount_point,'CSV_files/c3_instance_analysis_dataset_flyto_dcm.csv'), index=False)

# df_c3_flyto_mp4['pixel_spacing_x_mp4'] = df_c3_flyto_mp4['pixel_spacing_x_mp4']*10
# df_c3_flyto_mp4['pixel_spacing_y_mp4'] = df_c3_flyto_mp4['pixel_spacing_y_mp4']*10
# df_c3_flyto_mp4.to_csv(os.path.join(mount_point,'CSV_files/c3_instance_analysis_dataset_flyto_mp4.csv'), index=False)

# df_c3_flyto['file_path'] = df_c3_flyto['file_path'].str.replace('Groups/FAMLI/Restricted_access_data/Ultrasound/', '').str.replace('Groups/FAMLI/Deidentified_data/Ultrasound/', '').str.replace(f'Dataset_C3', f'Dataset_C3_masked_resampled_256_spc075').str.replace('.mp4', '.nrrd').str.replace('.dcm', '.nrrd')

# print(f"Total FlyTo train samples before filtering: {len(df_c3_flyto)}")
# df_c3_flyto_exists = df_c3_flyto[df_c3_flyto['file_path'].apply(lambda x: os.path.exists(os.path.join(mount_point, str(x))))]
# print(f"Total FlyTo train samples after filtering: {len(df_c3_flyto_exists)}")

# df_c3_flyto_exists.to_csv(os.path.join(mount_point,'CSV_files/c3_instance_analysis_dataset_flyto_train_masked_resampled_256_spc075.csv'), index=False)


# df_c3_flyto['file_path'].to_csv(os.path.join(mount_point,'CSV_files/c3_instance_analysis_dataset_flyto_train_masked_resampled_filepaths.txt'), index=False, header=False)